##ESPECIALIZAÇÃO EM ENGENHARIA DE DADOS 
#Projeto da disciplina Redes Neurais


O projeto de conclusão da disciplina de Redes Neurais, consiste na construção (e treinamento) de um modelo de classificador de imagens.

O objetivo é criar um classificador que classifique entre ao menos dois tipos de categorias (gatos e cachorros, cadeira e mesa, casa e carro por exemplo)

A única restrição é a necessidade da construção de um dataset  customizado


Equipe: Renato Lopes

Janeiro / 2022


#Montagem do Dataset

Para a construção do dataset customizado foi utilizado imagens de cards de terreno basico do jogo de cartas Magic the gathering.

As imagens foram baixadas do site https://mtgrex.com/?land=plains&land=island&land=swamp&land=mountain&land=forest&perPage=100 mas existem uma infinidade de sites onde é possivel baixar imagem de cartas do jogo

Para auxiliar no dowload das imagens foi utilizado o plugin Ftkun Batch Download image do google chrome.

Após o download das imagens as mesmas foram separadas em cinco categorias de terrenos basicos:

**Mountain - Swamp - Plains - Island - Florest**

Após a separação nos folders de cada categoria, foi criado o github com o dataset https://github.com/RenatoDLopes/MTG_Lands 

# Carregando o Dataset no colab


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import copy

In [ ]:
!rm -rf MTG_Lands

## Vide !rm --help para verificar todos os comando !rm
##!rm -rf : remove as bibliotecas selecionadas

In [ ]:
!git clone https://github.com/RenatoDLopes/MTG_Lands

#!git clone      Clone a git repository into a new directory

# split dataset

In [ ]:
import shutil
import os
from sklearn.model_selection import train_test_split

In [ ]:
base_dir = '/content/MTG_Lands'
base_dir_train = '/content/MTG_Lands/train'
base_dir_valid = '/content/MTG_Lands/valid'
os.makedirs(base_dir_train)
os.makedirs(base_dir_valid)

#os.makedirs(a) cria o diretorio a

In [ ]:
for land in ['Forest','Island','Mountain','Plains','Swamp']:
  land_files = os.listdir(os.path.join(base_dir,land))
  train, valid = train_test_split(land_files, test_size=0.2, random_state=1)
  os.makedirs(os.path.join(base_dir_train,land))
  os.makedirs(os.path.join(base_dir_valid,land))
  for archive in train:
    shutil.move(os.path.join(base_dir,land,archive), os.path.join(base_dir_train,land,archive))
  for archive in valid:
    shutil.move(os.path.join(base_dir,land,archive), os.path.join(base_dir_valid,land,archive))


#os.listdir(a) lê o diretorio a
#os.path.join(a,b) retorna o diretorio a/b
#os.makedirs(a) cria o diretorio a
#shutil.move (a,b) move o arquivo do diretorio a para o diretorio b

# experimentos

## Terrenos Basicos ('Forest','Island','Mountain','Plains','Swamp')

### Resnet - 3 canais (RGB)

In [ ]:
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

chosen_transforms = {'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean_nums, std_nums)
]), 'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean_nums, std_nums)
]),
}

In [ ]:
chosen_datasets = {x: datasets.ImageFolder(os.path.join(base_dir, x), chosen_transforms[x]) for x in ['train', 'valid']}

In [ ]:
chosen_datasets

In [ ]:
dataloaders = {x: torch.utils.data.DataLoader(chosen_datasets[x], batch_size=4,
  shuffle=True, num_workers=2)
              for x in ['train', 'valid']}

In [ ]:
dataset_sizes = {x: len(chosen_datasets[x]) for x in ['train', 'valid']}
class_names = chosen_datasets['train'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(dataset_sizes)
print(class_names)

In [ ]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([mean_nums])
    std = np.array([std_nums])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) 

In [ ]:
inputs, classes = next(iter(dataloaders['train']))

out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
res_mod = models.resnet34(pretrained=True)

num_ftrs = res_mod.fc.in_features
res_mod.fc = nn.Linear(num_ftrs, 5)

In [ ]:
res_mod

In [ ]:
for name, child in res_mod.named_children():
    print(name)

In [ ]:
res_mod = res_mod.to(device)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(res_mod.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=2):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            current_loss = 0.0
            current_corrects = 0

            # Here's where the training happens
            print('Iterating through data...')

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # We need to zero the gradients, don't forget it
                optimizer.zero_grad()

                # Time to carry out the forward training poss
                # We only need to log the loss stats if we are in training phase
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # We want variables to hold the loss statistics
                current_loss += loss.item() * inputs.size(0)
                current_corrects += torch.sum(preds == labels.data)

            epoch_loss = current_loss / dataset_sizes[phase]
            epoch_acc = current_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Make a copy of the model if the accuracy on the validation set has improved
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_since = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_since // 60, time_since % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # Now we'll load in the best model weights and return it
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def visualize_model(model, num_images=30):
    was_training = model.training
    model.eval()
    images_handeled = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['valid']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_handeled += 1
                fig = plt.figure()
                ax = fig.add_subplot(1, 2, 1)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_handeled == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
base_model = train_model(res_mod, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=1)
visualize_model(base_model)
plt.show()

In [ ]:
visualize_model(base_model)